In [52]:
import pandas as pd

training_df = pd.read_csv("./project/training_data.csv")
training_df


,title,category
0,The Three Amigos,None
1,Home Essentials Blue Floral Glass Vintage Styl...,Home & Kitchen
2,Cooper Wiring Quiet Toggle Switch Single Pole ...,Tools & Home Improvement
3,Baseboarders&reg; Wall Brackets,Tools & Home Improvement
4,The Great Wave Off Kanagawa Custom Rectangle M...,Office Products
...,...,...
20183,D0309-WML Leviton Decora Screwless Snap-On Wal...,None
20184,South Shore Axess Small 2-Door Storage Cabinet...,None
20185,Patriotic Flip Flop Ornament Assortment of 2,Home & Kitchen
20186,Cybrtrayd Life of the Party A116 Horse Chocola...,None


In [53]:
training_df['category'].value_counts()

None                        10123
Home & Kitchen               4960
Tools & Home Improvement     2080
Office Products              1144
Grocery & Gourmet Food       1102
Industrial & Scientific       588
Electronics                   191
Name: category, dtype: int64

In [54]:
import numpy as np
import random
none_indexes = training_df.index[training_df['category'] == "None"].tolist()
# print(len(none_indexes))
unselected_indexes = random.sample(none_indexes, 6000)
training_df.drop(training_df.index[unselected_indexes], inplace=True)
training_df['category'].value_counts()

Home & Kitchen              4960
None                        4123
Tools & Home Improvement    2080
Office Products             1144
Grocery & Gourmet Food      1102
Industrial & Scientific      588
Electronics                  191
Name: category, dtype: int64

In [55]:
import re
def normalize_text(s):
    s = s.lower()
    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    s = re.sub('\s\W', ' ', s)
    s = re.sub('\W\s', ' ', s)

    # make sure we didn't introduce any double spaces
    s = re.sub('\s+', ' ', s)
    return s

In [56]:
ip = list(map(normalize_text, training_df["title"]))
ip

['the three amigos',
 'home essentials blue floral glass vintage style water goblet set of 4',
 'cooper wiring quiet toggle switch single pole lighted 15 a 120 v ivory ul boxed',
 'baseboarders&reg wall brackets',
 'the great wave off kanagawa custom rectangle mouse pad oblong gaming mousepad in 220mm*180mm*3mm 9&quot;*7&quot; 912051',
 'nemcor pittsburgh penguins nhl 1 ultimate fan black/gold finger style throw cushion pillow',
 'souvnear 814614025003 silver color deer knob',
 '20 qty halco 50w mr16 fl 24v gu5.3 prism exn mr16exn/24v 50w 24v halogen flood lamp bulb',
 'rilakkuma bowl',
 'redbirdlinen 1pc brown sheer valance scarf 216&quot long beautiful elegant voile topper',
 'hospital bath towel 24x48in. 8 lb/dz pack of 12',
 'symphony in red and khaki by laurie maitland 29x29',
 'big train blended ice pumpkin spice 3.5 lbs bag)',
 'surpahs round 11 lb/5 kg digital kitchen and food scale w small removable bowl',
 'mikasa love story perfume bottle',
 '180 snacks nutty rice bites with

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
ip = vectorizer.fit_transform(ip)
ip

<14188x26671 sparse matrix of type '<class 'numpy.int64'>'
	with 156970 stored elements in Compressed Sparse Row format>

In [58]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
op = encoder.fit_transform(training_df["category"])
op

array([4, 2, 6, ..., 4, 2, 6])

In [59]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(ip, op, test_size=0.2)
x_train

<11350x26671 sparse matrix of type '<class 'numpy.int64'>'
	with 125299 stored elements in Compressed Sparse Row format>

In [60]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train, y_train)
print("Accuracy: ", nb.score(x_test, y_test))

Accuracy:  0.5500352360817478


In [13]:
def predict_category(index):
    return encoder.classes_[index]

In [14]:
def get_predicated(s):
    processed_ip = normalize_text(s)
    vectorized = vectorizer.transform([processed_ip])
    category = nb.predict(vectorized)
    return predict_category(category[0])

In [16]:
test_df = pd.read_csv('./project/testing_data.csv')
print(test_df[10:20])
final_data = pd.DataFrame(columns=["title", "predicted_category"])
final_data

                                                title  category
10  Scholastic Teacher's Friend 3-Column Chart Poc...       NaN
11  ALEKO FOPR36 Artificial 36 Inch Long Japanese ...       NaN
12     Garmin B60 600W Tilted Element 20&deg; - 8 Pin       NaN
13  Estwing E3-11 12-Ounce Drywall Hammer with Nyl...       NaN
14  ModHaus Diamond Button Tufted Clipped Corners ...       NaN
15  Yummy Grandmummy, Chihuahua with Pink Roses im...       NaN
16  Zorpia&reg; Laser Cut 50 X Wedding Candy Box W...       NaN
17  HP Officejet Pro L7780 Color All-in-One Printe...       NaN
18  Basix Deli Food Storage Container Lids 8 Ounce...       NaN
19  GOARTEA 50g (1.76 Oz) Organic Red Rosebud Rose...       NaN


,title,predicted_category


In [17]:
def add_predicted_data(title):
    final_data.loc[len(final_data.index)] = [title, get_predicated(title)]

In [18]:
for i in test_df["title"]:
    add_predicted_data(i)
    
final_data.to_csv('predicted_classes_with_none.csv')